In [7]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import time
import json
from ultralytics import YOLO
from PIL import Image


In [2]:
frame_path = Path("frames")

In [3]:
img = cv2.imread(str(frame_path / "2024_05_07_13_41_0.png"))

height, width, _ = img.shape
print(height, width)

350 800


In [4]:

weight_path = Path("train_results/best.pt")
frames_path = Path("frames")
model = YOLO(weight_path)

## test iterated frames for output

In [5]:
#iterating through the frames


i = 0
for frame in frames_path.iterdir():
    i += 1
    img = cv2.imread(str(frame))
    results = model(img)
    result = results[0]
    # print the boxes
    # print(result.boxes)
    xywhn = result.boxes.xywhn
    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        # if lower bound is below 40% of the image height, it's a parked car
        if y2 > 0.4 * height:
            # check in which third of the image width the center of the is
            if x < 0.33 * width:
                l = 1
                print("Left")
            elif x < 0.66 * width:
                print("Middle")
                c = 1
            else:
                print("Right")
                r = 1

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow("frame", img)
    cv2.waitKey(0)
  
    if i == 10:
        break

cv2.destroyAllWindows()


0: 352x800 2 parked-cars, 216.5ms
Speed: 2.0ms preprocess, 216.5ms inference, 1143.3ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 139.0ms
Speed: 17.0ms preprocess, 139.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 117.0ms
Speed: 1.0ms preprocess, 117.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 105.0ms
Speed: 1.0ms preprocess, 105.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 106.0ms
Speed: 0.0ms preprocess, 106.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 103.0ms
Speed: 1.0ms preprocess, 103.0ms inference, 0.0ms postprocess per image at shape (1, 3, 352, 800)
Middle
Right

0: 352x800 2 parked-cars, 103.0ms
Speed: 1.0ms preprocess, 103.0ms inference, 1.0ms postprocess per image at shape (1, 

## Check random frames for output

In [13]:
# random frames

frames = list(frames_path.iterdir())
random.shuffle(frames)
count = 20
save = True

for frame in frames:
    img = cv2.imread(str(frame))
    img_org = img.copy()
    results = model(img)
    result = results[0]
    xywhn = result.boxes.xywhn

    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        if y2 > 0.4 * height:
            if x < 0.33 * width:
                l = 1
            elif x < 0.66 * width:
                c = 1
            else:
                r = 1

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imshow(f'left:{l}, center: {c}, right: {r}', img)
    
    # if key pressed is n, print("copy")
    key = cv2.waitKey(0)
    if key == ord('n'):
        #copy the image to the to_label folder
        cv2.imwrite(f"to_label/{frame.name}", img_org)

    if save:
        im_bgr = result.plot()  # BGR-order numpy array
        im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image
        im_rgb.save(f"results/f2/{frame.name}")
        
    count -= 1
    cv2.destroyAllWindows()
    if count == 0:
        break





0: 352x800 2 parked-cars, 185.0ms
Speed: 1.0ms preprocess, 185.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 3 parked-cars, 210.0ms
Speed: 2.0ms preprocess, 210.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 3 parked-cars, 195.0ms
Speed: 3.0ms preprocess, 195.0ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 3 parked-cars, 193.3ms
Speed: 2.0ms preprocess, 193.3ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 2 parked-cars, 287.0ms
Speed: 6.0ms preprocess, 287.0ms inference, 2.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 3 parked-cars, 286.6ms
Speed: 10.0ms preprocess, 286.6ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 2 parked-cars, 290.1ms
Speed: 2.0ms preprocess, 290.1ms inference, 1.0ms postprocess per image at shape (1, 3, 352, 800)

0: 352x800 2 parked-cars, 198.0ms
Speed: 1.0ms preprocess, 198.0ms 

## Proper function to jsonify output

In [6]:
def frame_to_json(frame, verbose=False, id = None):
    img = cv2.imread(str(frame))
    results = model(img, verbose=verbose)
    result = results[0]
    # print the boxes
    # print(result.boxes)
    xywhn = result.boxes.xywhn
    # display the image
    #plot xy points on the image
    l, r, c = 0, 0, 0
    for xywh in xywhn:
        x, y, w, h = xywh.cpu().numpy()
        x, y, w, h = x * width, y * height, w * width, h * height
        x1, y1, x2, y2 = int(x - w/2), int(y - h/2), int(x + w/2), int(y + h/2)

        # if lower bound is below 35% of the image height, it's a parked car
        if y2 > 0.35 * height:
            # check in which third of the image width the car is
            if x < 0.33 * width:
                l = 1
            elif x < 0.66 * width:
                c = 1
            else:
                r = 1

    if verbose:
        print(f"Left: {l}, Right: {r}, Center: {c}")
    name_split = frame.name.split("_")
    if len(name_split) < 6:
        return None
    year, month, day, hour, minute, second = name_split[:6]
    second = str(int(second.split(".")[0]) * 10)
    return {
        "id": id,
        "name": frame.name,
        "year": year,
        "month": month,
        "day": day,
        "hour": hour,
        "minute": minute,
        "second": second,
        "left": l,
        "right": r,
        "center": c
    }

In [7]:
count = 0
for frame in frames_path.iterdir():
    json_reult = frame_to_json(frame)
    print(json_reult)
    count += 1
    if count == 10:
        break

{'id': None, 'name': '2024_05_07_13_41_0.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '0', 'left': 0, 'right': 1, 'center': 1}
{'id': None, 'name': '2024_05_07_13_41_1.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '10', 'left': 0, 'right': 1, 'center': 1}
{'id': None, 'name': '2024_05_07_13_41_2.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '20', 'left': 0, 'right': 1, 'center': 1}
{'id': None, 'name': '2024_05_07_13_41_3.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '30', 'left': 0, 'right': 1, 'center': 1}
{'id': None, 'name': '2024_05_07_13_41_4.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '40', 'left': 0, 'right': 1, 'center': 1}
{'id': None, 'name': '2024_05_07_13_41_5.png', 'year': '2024', 'month': '05', 'day': '07', 'hour': '13', 'minute': '41', 'second': '50',

In [8]:
# iterate through all the frames and save the results in a json file in results folder
# speed about 9 frames per second
#1.5h to process the whole folder, only to do on final model
results = []
i = 0
for frame in frames_path.iterdir():
    json_reult = frame_to_json(frame, id=i)
    results.append(json_reult)
    i += 1

    ############################################
    # break after 100 frames
    # if i == 100:
    #     break
    ############################################
    
    if i % 1000 == 0:
        print(f"Processed {i} frames")


with open("results/results.json", "w") as f:
    json.dump(results, f)

    

Processed 1000 frames
Processed 2000 frames
Processed 3000 frames
Processed 4000 frames
Processed 5000 frames
Processed 6000 frames
Processed 7000 frames
Processed 8000 frames
Processed 9000 frames
Processed 10000 frames
Processed 11000 frames
Processed 12000 frames
Processed 13000 frames
Processed 14000 frames
Processed 15000 frames
Processed 16000 frames
Processed 17000 frames
Processed 18000 frames
Processed 19000 frames
Processed 20000 frames
Processed 21000 frames
Processed 22000 frames
Processed 23000 frames
Processed 24000 frames
Processed 25000 frames
Processed 26000 frames
Processed 27000 frames
Processed 28000 frames
Processed 29000 frames
Processed 30000 frames
Processed 31000 frames
Processed 32000 frames
Processed 33000 frames
Processed 34000 frames
Processed 35000 frames
Processed 36000 frames
Processed 37000 frames
Processed 38000 frames
Processed 39000 frames
Processed 40000 frames
Processed 41000 frames
Processed 42000 frames
Processed 43000 frames
Processed 44000 fram

In [9]:
# prettifing the json file
with open("results/results.json", "r") as f:
    results = json.load(f)

with open("results/results_pretty.json", "w") as f:
    json.dump(results, f, indent=4)

    